In [1]:
import numpy as np
import os
from mxnet import nd, gpu, gluon, autograd
import mxnet as mx
import pandas as pd
from mxnet import gluon
import gc 

from gluonts.env import env

# env._push(use_tqdm=False)

import json
from gluonts.dataset.common import load_datasets
from gluonts.dataset.repository.datasets import get_dataset
from gluonts.model.seq2seq import MQRNNEstimator
from gluonts.mx.trainer import Trainer
from gluonts.evaluation import Evaluator
from pathlib import Path
from gluonts.evaluation.backtest import make_evaluation_predictions
from tqdm.notebook import tqdm


from gluonts.env import env

/home/pedro/miniconda/envs/mxnet_clean/lib/python3.7/site-packages/gluonts/json.py:46: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  "Using `json`-module for json-handling. "


In [2]:
# activate whatever environment. 
# cd into the gluonts-local directory
# pip install . 

In [3]:
from quantile_forecast import *

## Below we train the model. If you use the bash script instead, the training happens in a couple of minutes, but it takes much longer here.

In [15]:
dataset_path = "/home/pedro/pedro_code/deep-traffic/fresh/data/data_to_tune/chic/chic.gluonts"
model_path = "./models/chic_mqrnn"
config_path = "./configs/test/config_chic_5_monot.json"
quantile_boundary= 1e-3
monotonic = True

with open(config_path) as json_file:
        config = json.load(json_file)
        print(config)
        
num_quantiles = config["num_quantiles"]
dataset = load_dataset(dataset_path)
quantiles = make_quantiles(dataset, config["num_quantiles"], quantile_boundary)
trainer = make_trainer(config)


estimator = make_mqrnn_estimator(trainer, quantiles, monotonic, dataset.metadata.freq, prediction_length = 1)
predictor = make_predictor(estimator, dataset)

metrics, _, _ = evaluate(predictor, dataset, quantiles, prediction_length = 1)

print("All Test Data: ")
print(metrics)

{'epochs': 334, 'learning_rate': 0.00020122811163700603, 'learning_rate_decay_factor': 0.283951731448596, 'weight_decay': 8.564701478093447e-09, 'num_quantiles': 65}
learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.


/home/pedro/miniconda/envs/mxnet_clean/lib/python3.7/site-packages/gluonts/transform/split.py:36: FutureWarning: Timestamp.freq is deprecated and will be removed in a future version
  return _shift_timestamp_helper(ts, ts.freq, offset)
/home/pedro/miniconda/envs/mxnet_clean/lib/python3.7/site-packages/gluonts/transform/feature.py:352: FutureWarning: Timestamp.freq is deprecated and will be removed in a future version
  self._min_time_point, self._max_time_point, freq=start.freq
/home/pedro/miniconda/envs/mxnet_clean/lib/python3.7/site-packages/gluonts/model/forecast.py:502: FutureWarning: The 'freq' argument in Timestamp is deprecated and will be removed in a future version.
  self.start_date = pd.Timestamp(start_date, freq=freq)


  0%|          | 0/128 [00:00<?, ?it/s]

All Test Data: 
{'crps': 0.009828123630566347, '60cover': 0.53125, '80cover': 0.7109375, '95cover': 0.953125}
